In [1]:
%matplotlib widget
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import json
import os
import copy

# Fix cut off xlabel
# https://stackoverflow.com/a/17390833
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})


In [2]:
def get_list_of_seeds(subchunk_freq_json_path):
    with open(subchunk_freq_json_path) as json_file:
        data = json.load(json_file)
    seeds = [seed for [seed, _counts] in data["per_seed"]]
    return seeds

In [3]:
def plot_freq_block(data, block_name, filename):
    H = np.zeros((16, 16))
    H += np.array_split(data[block_name], 16)
    return plot_freq(H, block_name, filename)

def plot_freq_block_sum(data, block_names, filename):
    H = np.zeros((16, 16))
    for block_name in block_names:
        try:
            H += np.array_split(data[block_name], 16)
        except KeyError:
            pass
    block_name = "+".join(block_names)
    return plot_freq(H, block_name, filename)

def plot_freq(H, block_name, filename):    
    print(f"{block_name} mean: {np.mean(H)}, stddev: {np.std(H)}")
    move_to_diamonds = move_to_diamonds_msg(H)

    #plt.imshow(H, interpolation='none')
    #plt.show()
    fig = plt.figure(figsize=(6, 4))

    # https://stackoverflow.com/a/16492880
    ax = fig.add_subplot(111)
    ax.set_title(f'{block_name} frequency\n(subchunk coordinates, centered at diamond ore at (8, 8))\n{move_to_diamonds}', fontdict={'fontsize': 10})
    plt.xlabel('x coordinate')
    plt.ylabel('z coordinate')
    
    # I want to differentiate 0 from low values
    # https://stackoverflow.com/a/22552651
    # mask some 'bad' data, in your case you would have: data == 0
    H = np.ma.masked_where(H == 0, H)
    cmap = copy.copy(matplotlib.cm.get_cmap("viridis"))
    cmap.set_bad(color='black')

    plt.imshow(H, interpolation='none', origin='upper', cmap = cmap)
    ax.set_aspect('equal')
    cax = fig.add_axes([0.12, 0.1, 0.78, 0.7])
    cax.get_xaxis().set_visible(False)
    cax.get_yaxis().set_visible(False)
    cax.patch.set_alpha(0)
    cax.set_frame_on(False)
    plt.colorbar(orientation='vertical')
    plt.show()
    # Ensure dir exists
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    plt.savefig(filename)
    plt.close()

def sum_of_8_neighbors_and_center(H, idx):
    [i, j] = idx
    min_i = i-1
    min_j = j-1
    max_i = i+2
    max_j = j+2
    # https://stackoverflow.com/a/21396821
    nei = np.take(H, range(min_i,max_i),mode='wrap', axis=0).take(range(min_j,max_j),mode='wrap',axis=1)
    return np.sum(nei)

def sum_of_24_neighbors_and_center(H, idx):
    [i, j] = idx
    min_i = i-2
    min_j = j-2
    max_i = i+3
    max_j = j+3
    # https://stackoverflow.com/a/21396821
    nei = np.take(H, range(min_i,max_i),mode='wrap', axis=0).take(range(min_j,max_j),mode='wrap',axis=1)
    return np.sum(nei)

def move_to_diamonds_msg(H):
    z, x = np.where(H == np.amax(H))
    # in case of tie, take the one at index 0
    x = x[0]
    z = z[0]
    # The probability is calculated as sum_of_8_neighbors_of_max_value / sum_of_all_values
    #prob = np.amax(H) / np.sum(H)
    if np.sum(H) == 0:
        prob_1 = 0.0
        prob_2 = 0.0
    else:
        prob_1 = sum_of_8_neighbors_and_center(H, [z, x]) / np.sum(H)
        prob_2 = sum_of_24_neighbors_and_center(H, [z, x]) / np.sum(H)
    move_to_str = f'To find diamonds move '
    if 8 - x < 0:
        move_to_str += f'{x - 8} blocks west (-x)'
    else:
        move_to_str += f'{8 - x} blocks east (+x)'
    move_to_str += ' and '
    if 8 - z < 0:
        move_to_str += f'{z - 8} blocks north (-z)'
    else:
        move_to_str += f'{8 - z} blocks south (+z)'
    move_to_str += f'.\n{(8-x, 8-z)}, prob_1={prob_1:.03f}, prob_2={prob_2:.03f}'
    return move_to_str

In [4]:
subchunk_freq_json_path = 'subchunk_freq_centered_at_diamond.json'
seeds = get_list_of_seeds(subchunk_freq_json_path)
print("Data obtained from the average of a 2000x2000 block area from these seeds:")
print(seeds)

with open(subchunk_freq_json_path) as json_file:
    data = json.load(json_file)



Data obtained from the average of a 2000x2000 block area from these seeds:
[3916027865427300396, -7151032874728305412, -7296285796647183684, -420246996566605359, -3080015041417313864, -804827390349354441, -3071541673020963785, -4659182379820619264, 7177585373950549858, 4357937157405796422, -4412356096731891337, -8500107817075736912, 797383349100663716, 7200556082090718124, -8989667791662813116, 1474710366174659160, -4465786308719816483, -558489853248530916, -5450423930667436192, -6686265662720560062, -6833148301086874319, -9180918114083235885, -3208759163418100650, -8836411830940369684, 7868637189286194290, -2672478081932664491, -9128037715974859558, 4059329449073125427, -28306679728284652, -8647947548211764288, -1013960319343502453, 6978393985624498672, -189474080393998282, 277075514134440699, -1043574159800086966, 2426144334969791549, -2515501214971595370, 8940809639103610494, 186304078426175181, -6126059897333184392, -697417934091828864, -159034789280475321, 907185561593286697, -402

In [10]:
def plot_ores(data, output_dir='plots_centered_at_diamond/ores/'):
    # Plot ores by adding ore + deepslate_ore
    plot_freq_block_sum(data, ['minecraft:coal_ore', 'minecraft:deepslate_coal_ore'], f'{output_dir}/coal_subchunk_freq.png')
    plot_freq_block_sum(data, ['minecraft:copper_ore', 'minecraft:deepslate_copper_ore'], f'{output_dir}/copper_subchunk_freq.png')
    plot_freq_block_sum(data, ['minecraft:diamond_ore', 'minecraft:deepslate_diamond_ore'], f'{output_dir}/diamond_subchunk_freq.png')
    plot_freq_block_sum(data, ['minecraft:emerald_ore', 'minecraft:deepslate_emerald_ore'], f'{output_dir}/emerald_subchunk_freq.png')
    plot_freq_block_sum(data, ['minecraft:gold_ore', 'minecraft:deepslate_gold_ore'], f'{output_dir}/gold_subchunk_freq.png')
    plot_freq_block_sum(data, ['minecraft:iron_ore', 'minecraft:deepslate_iron_ore'], f'{output_dir}/iron_subchunk_freq.png')
    plot_freq_block_sum(data, ['minecraft:lapis_ore', 'minecraft:deepslate_lapis_ore'], f'{output_dir}/lapis_subchunk_freq.png')
    plot_freq_block_sum(data, ['minecraft:redstone_ore', 'minecraft:deepslate_redstone_ore'], f'{output_dir}/redstone_subchunk_freq.png')

plot_ores(data['total'])

In [32]:
for seed, x in data['per_seed']:
    plot_ores(x, f'plots_centered_at_diamond/seeds/{seed}/ores/')

minecraft:coal_ore+minecraft:deepslate_coal_ore mean: 10294.34375, stddev: 1678.6098575043986


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

minecraft:copper_ore+minecraft:deepslate_copper_ore mean: 1822.9140625, stddev: 1463.4409634666667


<ipython-input-29-c693d7931bce>:48: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.savefig(filename)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

minecraft:lapis_ore+minecraft:deepslate_lapis_ore mean: 205.46875, stddev: 1145.5849654591045


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

minecraft:redstone_ore+minecraft:deepslate_redstone_ore mean: 1469.07421875, stddev: 1060.6622371001652


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
total = data['total']
for block_name, counts in total.items():
    plot_freq_block(total, block_name, f'plots_centered_at_diamond/{block_name}_subchunk_freq.png')

minecraft:acacia_door mean: 1.8515625, stddev: 1.9022595271396987


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

minecraft:acacia_fence mean: 9.5, stddev: 4.537172853220384


<ipython-input-29-c693d7931bce>:48: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.savefig(filename)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

minecraft:yellow_terracotta mean: 3573.50390625, stddev: 47.29305231470275


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

minecraft:yellow_wool mean: 1.01953125, stddev: 0.9820704049473427


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
subchunk_freq_all = data
subchunk_freq_json_path = 'subchunk_freq_centered_at_diamond_odd_chunks.json'
seeds = get_list_of_seeds(subchunk_freq_json_path)

with open(subchunk_freq_json_path) as json_file:
    data = json.load(json_file)


In [8]:
def calculate_even_chunks_data_given_all_and_odd_chunks(all_chunks, odd_chunks):
    # structure: {
    # per_seed: [[seed, {block_id: counts}]]
    # total: {block_id: counts}
    # }
    def subtract_counts(all_counts, odd_counts):
        even_counts = copy.deepcopy(all_counts)
        for i, x in enumerate(odd_counts):
            even_counts[i] -= x
        return even_counts

    def seed_idx(data, target_seed):
        i = 0
        for i, (seed, per_seed) in enumerate(data):
            if seed == target_seed:
                return i
        return None
    
    even_chunks = copy.deepcopy(all_chunks)
    
    for block_name, counts in odd_chunks['total'].items():
        even_chunks['total'][block_name] = subtract_counts(all_chunks['total'][block_name], counts)
    
    for i, (seed, per_seed) in enumerate(odd_chunks['per_seed']):
        even_seed_idx = seed_idx(even_chunks['per_seed'], seed)
        #odd_seed_idx = seed_idx(odd_chunks['per_seed'], seed)
        all_seed_idx = seed_idx(all_chunks['per_seed'], seed)
        for block_name, counts in per_seed.items():
            even_chunks['per_seed'][even_seed_idx][1][block_name] = subtract_counts(all_chunks['per_seed'][all_seed_idx][1][block_name], counts)
            
    return even_chunks

data_odd = data
data_even = calculate_even_chunks_data_given_all_and_odd_chunks(subchunk_freq_all, data_odd)

In [11]:
plot_ores(data_even['total'], output_dir='plots_centered_at_diamond/even_chunks/ores/')
plot_ores(data_odd['total'], output_dir='plots_centered_at_diamond/odd_chunks/ores/')

minecraft:coal_ore+minecraft:deepslate_coal_ore mean: 213776.44921875, stddev: 26115.651047562384


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

minecraft:copper_ore+minecraft:deepslate_copper_ore mean: 37100.40234375, stddev: 19900.6162413966


<ipython-input-3-c693d7931bce>:48: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.savefig(filename)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

minecraft:lapis_ore+minecraft:deepslate_lapis_ore mean: 4914.48828125, stddev: 20848.87698414144


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

minecraft:redstone_ore+minecraft:deepslate_redstone_ore mean: 35040.4609375, stddev: 21101.18140849966


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
for seed, x in data_even['per_seed']:
    plot_ores(x, f'plots_centered_at_diamond/even_chunks/seeds/{seed}/ores/')
    
for seed, x in data_odd['per_seed']:
    plot_ores(x, f'plots_centered_at_diamond/odd_chunks/seeds/{seed}/ores/')

minecraft:coal_ore+minecraft:deepslate_coal_ore mean: 5144.34375, stddev: 836.5874313384331


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

minecraft:copper_ore+minecraft:deepslate_copper_ore mean: 913.88671875, stddev: 727.6778662369142


<ipython-input-3-c693d7931bce>:48: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.savefig(filename)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

minecraft:lapis_ore+minecraft:deepslate_lapis_ore mean: 102.5390625, stddev: 573.4391505745149


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

minecraft:redstone_ore+minecraft:deepslate_redstone_ore mean: 731.72265625, stddev: 530.6252992806641


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …